In [22]:
import IPython
import numpy as np
from ipywidgets import Image
#
from Bio.PDB import PDBParser, PDBIO
#
import openmm
import openmm.app
from openmm.app import PDBFile
from openmm.app import CheckpointReporter
#
from openff.toolkit import ForceField, Molecule, Topology
from openff.units import Quantity, unit
from openff.pablo import CCD_RESIDUE_DEFINITION_CACHE
from openff.pablo import ResidueDefinition
from openff.pablo.chem import PEPTIDE_BOND
from openff.pablo import topology_from_pdb
from openff.interchange import Interchange
from openff.interchange.components._packmol import (
    RHOMBIC_DODECAHEDRON,
    solvate_topology,
)
import sys
sys.path.append('..')
from ptm_prototype import draw_molecule
from ptm_prototype import react_mod
from ptm_prototype import parametrize_with_nagl
#
import rdkit
from rdkit import Chem
from rdkit import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem.rdChemReactions import ReactionFromSmarts
from rdkit.Chem.rdchem import AtomPDBResidueInfo
Chem.SetDefaultPickleProperties(Chem.PropertyPickleOptions.AllProps)
draw_options = Chem.Draw.MolDrawOptions()
draw_options.addAtomIndices = True
print(rdkit.__version__)

2025.03.6


In [23]:
cysteine = CCD_RESIDUE_DEFINITION_CACHE["CYS"][0].to_openff_molecule()
Elironrasib = Molecule.from_smiles("C[C@H](OC)C(N=CC=C1)=[C@@]1[C@](N2CC)=C(CC(C)(C)COC([C@H]3NN(C([C@@H](NC([C@H](C(C)C)N(C(C4(F)CCN(C(C#CC(N(C)C)(C)C)=O)CC4)=O)C)=O)C[C@H]5CN6CCO5)=O)CCC3)=O)C7=C2C=CC6=C7")
Elironrasib.generate_unique_atom_names()
#
covalent_triple_bond_smarts_cis = '[C:10]-[S:1]-[H:2].[C:11]-[C:3]#[C:4]-[C:5](=[O:6])-[N:7](-[C:8])-[C:9]>>[C:10]-[S:1]/[C:3](-[C:11])=[C:4](/[H])[C:5](=[O:6])-[N:7](-[C:8])-[C:9]'
rxn = ReactionFromSmarts(covalent_triple_bond_smarts_cis)
products = list(react_mod([cysteine, Elironrasib], covalent_triple_bond_smarts_cis))
adduct = products[0][0]
adduct.atom(5).name = 'H79x'
#
ligand_resdef = ResidueDefinition.from_molecule(
    molecule=adduct,
    residue_name="LIG",
    linking_bond=PEPTIDE_BOND,
)
#
ser_resdef = CCD_RESIDUE_DEFINITION_CACHE["SER"][6]
sen_resdef = ser_resdef.replace(residue_name='SEN', description='SERINE deprotonated')
#
thr_resdef = CCD_RESIDUE_DEFINITION_CACHE["THR"][6]
thn_resdef = thr_resdef.replace(residue_name='THN', description='THREONINE deprotonated')
#
residue_database_1 = CCD_RESIDUE_DEFINITION_CACHE.with_({"LIG": [ligand_resdef]})
residue_database_2 = residue_database_1.with_({"SEN": [sen_resdef]})
residue_database_3 = residue_database_2.with_({"THN": [thn_resdef]})
#
covalent_topology = topology_from_pdb(
    "cofold/final_covalent_mod.pdb",
    residue_database=residue_database_3,
)
covalent_topology.box_vectors=None
#
force_field = ForceField("openff-2.2.1.offxml", "ff14sb_off_impropers_0.0.4.offxml")
#
charge_handler = force_field.get_parameter_handler('LibraryCharges')
mg_charge_params = {
    "smirks": "[#12+2:1]",
    "charge1": 2.0 * unit.elementary_charge
}
charge_handler.add_parameter(mg_charge_params)
#
vdw_handler = force_field.get_parameter_handler('vdW')
vdw_params = {
    'smirks': '[#12+2:1]',
    'epsilon': 0.07 * unit.kilocalorie_per_mole,
    'sigma': 1.7 * unit.angstrom,
}
vdw_handler.add_parameter(vdw_params)
#
MG = Molecule.from_pdb_and_smiles('cofold/MG.pdb', '[#12+2:1]')
covalent_topology.add_molecule(MG)
#
rdmol = Chem.SDMolSupplier('cofold/GNP_H_charged.sdf')[0]
Chem.SanitizeMol(rdmol)
GNP = Molecule.from_rdkit(rdmol, allow_undefined_stereo=True)
GNP.name = "GNP"
for atom in GNP.atoms:
    atom.metadata['residue_name'] = 'GNP'
covalent_topology.add_molecule(GNP)
#
final_topology = solvate_topology(
    covalent_topology,
    nacl_conc=Quantity(0.1, "mol/L"),
    padding=Quantity(1.0, "nm"),
    box_shape=RHOMBIC_DODECAHEDRON,
)
#
interchange = parametrize_with_nagl(
    force_field=force_field,
    topology=final_topology,
)
interchange.to_pdb('cofold/cofold.pdb')

[00:10:31] mapped atoms in the reactants were not mapped in the products.
  unmapped numbers are: 2 


adding dummy charges to force field ...
making Interchange ...
replacing dummy charges with NAGL charges ... 
assigning graph charges to C904H1418FN241O277S5 ...


[00:11:32] WARNING: Proton(s) added/removed



continuing with dummy charge replacement ...
assigning graph charges to GNP ...


[00:11:39] WARNING: Proton(s) added/removed



continuing with dummy charge replacement ...


In [24]:
def swap_pdb_chains(input_pdb_file, output_pdb_file):
    """
    Reads a PDB file and swaps the chain identifiers 'A' and 'B'.
    All other chain identifiers are left unchanged.

    Args:
        input_pdb_file (str): Path to the original PDB file.
        output_pdb_file (str): Path to save the modified PDB file.
    """
    modified_lines = []

    try:
        with open(input_pdb_file, 'r') as infile:
            for line in infile:
                # Standard PDB lines with chain ID: ATOM, HETATM, ANISOU, SIGATM, SIGUIJ, TER
                # We check the record name (first 6 characters)
                record_name = line[:6].strip()

                # Chain ID is at column 22 (index 21 in Python) for these records
                if record_name in ('ATOM', 'HETATM', 'ANISOU', 'SIGATM', 'SIGUIJ', 'TER'):
                    chain_id = line[21] # Get the current chain ID

                    # Determine the new chain ID
                    new_chain_id = chain_id
                    if chain_id == 'C':
                        new_chain_id = 'A'

                    # Construct the new line by splicing
                    # [0:21] is the part before the chain ID
                    # [22:] is the part after the chain ID
                    modified_line = line[:21] + new_chain_id + line[22:]
                    modified_lines.append(modified_line)
                else:
                    # Keep all other lines (HEADER, REMARK, END, etc.) unchanged
                    modified_lines.append(line)

    except FileNotFoundError:
        print(f"Error: File not found at {input_pdb_file}")
        return

    # Write the modified lines to the output file
    with open(output_pdb_file, 'w') as outfile:
        outfile.writelines(modified_lines)

swap_pdb_chains('crystal/final_covalent_mod_1-processed.pdb', 'crystal/final_covalent_mod_1-processed_swapped.pdb')

!pymol sort A and B chains

In [25]:
cysteine = CCD_RESIDUE_DEFINITION_CACHE["CYS"][0].to_openff_molecule()
Elironrasib = Molecule.from_smiles("C[C@H](OC)C(N=CC=C1)=[C@@]1[C@](N2CC)=C(CC(C)(C)COC([C@H]3NN(C([C@@H](NC([C@H](C(C)C)N(C(C4(F)CCN(C(C#CC(N(C)C)(C)C)=O)CC4)=O)C)=O)C[C@H]5CN6CCO5)=O)CCC3)=O)C7=C2C=CC6=C7")
Elironrasib.generate_unique_atom_names()
#
covalent_triple_bond_smarts_cis = '[C:10]-[S:1]-[H:2].[C:11]-[C:3]#[C:4]-[C:5](=[O:6])-[N:7](-[C:8])-[C:9]>>[C:10]-[S:1]/[C:3](-[C:11])=[C:4](/[H])[C:5](=[O:6])-[N:7](-[C:8])-[C:9]'
rxn = ReactionFromSmarts(covalent_triple_bond_smarts_cis)
products = list(react_mod([cysteine, Elironrasib], covalent_triple_bond_smarts_cis))
adduct = products[0][0]
adduct.atom(5).name = 'H79x'
#
ligand_resdef = ResidueDefinition.from_molecule(
    molecule=adduct,
    residue_name="LIG",
    linking_bond=PEPTIDE_BOND,
)
#
ser_resdef = CCD_RESIDUE_DEFINITION_CACHE["SER"][6]
sen_resdef = ser_resdef.replace(residue_name='SEN', description='SERINE deprotonated')
#
thr_resdef = CCD_RESIDUE_DEFINITION_CACHE["THR"][6]
thn_resdef = thr_resdef.replace(residue_name='THN', description='THREONINE deprotonated')
#
residue_database_1 = CCD_RESIDUE_DEFINITION_CACHE.with_({"LIG": [ligand_resdef]})
residue_database_2 = residue_database_1.with_({"SEN": [sen_resdef]})
residue_database_3 = residue_database_2.with_({"THN": [thn_resdef]})
#
covalent_topology = topology_from_pdb(
    "crystal/final_covalent_mod_1-processed_swapped_pymol.pdb",
    residue_database=residue_database_3,
)
covalent_topology.box_vectors=None
#
force_field = ForceField("openff-2.2.1.offxml", "ff14sb_off_impropers_0.0.4.offxml")
#
charge_handler = force_field.get_parameter_handler('LibraryCharges')
mg_charge_params = {
    "smirks": "[#12+2:1]",
    "charge1": 2.0 * unit.elementary_charge
}
charge_handler.add_parameter(mg_charge_params)
#
vdw_handler = force_field.get_parameter_handler('vdW')
vdw_params = {
    'smirks': '[#12+2:1]',
    'epsilon': 0.07 * unit.kilocalorie_per_mole,
    'sigma': 1.7 * unit.angstrom,
}
vdw_handler.add_parameter(vdw_params)
#
MG = Molecule.from_pdb_and_smiles('crystal/MG.pdb', '[#12+2:1]')
covalent_topology.add_molecule(MG)
#
rdmol = Chem.SDMolSupplier('crystal/GNP_H_charged.sdf')[0]
Chem.SanitizeMol(rdmol)
GNP = Molecule.from_rdkit(rdmol, allow_undefined_stereo=True)
GNP.name = "GNP"
for atom in GNP.atoms:
    atom.metadata['residue_name'] = 'GNP'
covalent_topology.add_molecule(GNP)
#
final_topology = solvate_topology(
    covalent_topology,
    nacl_conc=Quantity(0.1, "mol/L"),
    padding=Quantity(1.0, "nm"),
    box_shape=RHOMBIC_DODECAHEDRON,
)
#
interchange_target = parametrize_with_nagl(
    force_field=force_field,
    topology=final_topology,
)
interchange_target.to_pdb('crystal/crystal.pdb')

[00:11:47] mapped atoms in the reactants were not mapped in the products.
  unmapped numbers are: 2 


adding dummy charges to force field ...
making Interchange ...
replacing dummy charges with NAGL charges ... 
assigning graph charges to C904H1418FN241O277S5 ...


[00:12:47] WARNING: Proton(s) added/removed



continuing with dummy charge replacement ...
assigning graph charges to GNP ...


[00:12:58] WARNING: Proton(s) added/removed



continuing with dummy charge replacement ...


In [26]:
import openmm.openmm as openmm
from openmm import app
from openff.units.openmm import to_openmm
import openmm as openmm
from openmm import unit

def interchange_solute_reposition_and_minimise(interchange, interchange_target):

    system = interchange.to_openmm_system()
    topology = interchange.to_openmm_topology()
    system_positions = to_openmm(interchange.positions)
    print(f"Total number of atoms in system: {len(system_positions)}")
    system.addForce(
        openmm.MonteCarloBarostat(
            1*unit.atmosphere, 
            300*unit.kelvin,
            ))
    integrator = openmm.LangevinIntegrator(
        300*unit.kelvin, 
        1.0/unit.picosecond, 
        0.002*unit.picoseconds
    )
    simulation_cofold = app.Simulation(topology, system, integrator)
    simulation_cofold.context.setPositions(system_positions)
    simulation_cofold.minimizeEnergy()
    state_cofold = simulation_cofold.context.getState(getPositions=True)
    with open(f'cofold_min.pdb', 'w') as output:
        PDBFile.writeFile(topology, state_cofold.getPositions(), output)
    #
    solute_atom_indices = []
    EXCLUDED_RESIDUES = {'HOH', 'NA', 'CL'}    
    for atom in topology.atoms():
        if atom.residue.name not in EXCLUDED_RESIDUES:
            solute_atom_indices.append(atom.index)            
    print(f"Number of solute atoms identified: {len(solute_atom_indices)}")
    #
    target_positions = to_openmm(interchange_target.positions)
    print(f"Number of atoms in target system: {len(target_positions)}")
    #
    full_reference_values = system_positions.value_in_unit(unit.nanometer) 
    target_values = target_positions.value_in_unit(unit.nanometer)
    target_values_restrained = target_values[solute_atom_indices]
    for i, atom_idx in enumerate(solute_atom_indices):
        full_reference_values[atom_idx] = target_values_restrained[i]
    full_reference_positions_openmm = unit.Quantity(full_reference_values, unit.nanometer)
    print(f"Final number of reference positions prepared: {len(full_reference_positions_openmm)}")
    #
    K_RESTRAINT = 100.0 * unit.kilojoules_per_mole / unit.nanometer**2
    restraint_force = openmm.CustomExternalForce('0.5 * K * ((x-x0)^2 + (y-y0)^2 + (z-z0)^2)')
    restraint_force.addGlobalParameter('K', K_RESTRAINT)
    restraint_force.addPerParticleParameter('x0')
    restraint_force.addPerParticleParameter('y0')
    restraint_force.addPerParticleParameter('z0')    
    for i in solute_atom_indices:
        pos_quantity = full_reference_positions_openmm[i]
        ref_x, ref_y, ref_z = pos_quantity.value_in_unit(unit.nanometer)
        restraint_force.addParticle(i, [ref_x, ref_y, ref_z])        
    res_force_int = system.addForce(restraint_force)            
    #
    integrator = openmm.LangevinIntegrator(
        300*unit.kelvin, 
        1.0/unit.picosecond, 
        0.002*unit.picoseconds
    )
    simulation_crystal_res = app.Simulation(topology, system, integrator)
    simulation_crystal_res.context.setPositions(full_reference_positions_openmm)
    simulation_crystal_res.minimizeEnergy()
    state_crystal_res = simulation_crystal_res.context.getState(getPositions=True)
    #
    system.removeForce(res_force_int)
    integrator = openmm.LangevinIntegrator(
        300*unit.kelvin, 
        1.0/unit.picosecond, 
        0.002*unit.picoseconds
    )
    simulation_crystal = app.Simulation(topology, system, integrator)
    simulation_crystal.context.setPositions(state_crystal_res.getPositions())
    simulation_crystal.minimizeEnergy()
    state_crystal = simulation_crystal.context.getState(getPositions=True)
    with open(f'crystal_min.pdb', 'w') as output:
        PDBFile.writeFile(topology, state_crystal.getPositions(), output)
    
    return simulation_cofold, simulation_crystal

simulation_cofold, simulation_crystal = interchange_solute_reposition_and_minimise(interchange, interchange_target)

Total number of atoms in system: 54649
Number of solute atoms identified: 5380
Number of atoms in target system: 56250
Final number of reference positions prepared: 54649


In [27]:
'''
integrator = openmm.LangevinIntegrator(
    300*unit.kelvin, 
    1.0/unit.picosecond, 
    0.002*unit.picoseconds
)
for i in range(2,3):
    integrator.setRandomNumberSeed(i)
    simulation_crystal.context.setPositions(PDBFile('long_test/crystal_min.pdb').getPositions())
    simulation_crystal.context.setVelocitiesToTemperature(300*openmm.unit.kelvin)
    simulation_crystal.reporters.clear()
    simulation_crystal.reporters.append(app.DCDReporter(f'long_test/crystal_{i}.dcd', 1000))
    simulation_crystal.reporters.append(app.StateDataReporter(f'long_test/crystal_{i}.csv', 1000, 
                                step=True, potentialEnergy=True, kineticEnergy=True, 
                                temperature=True, totalEnergy=True, volume=True, density=True, speed=True))
    simulation_crystal.step(25000000)
'''

"\nintegrator = openmm.LangevinIntegrator(\n    300*unit.kelvin, \n    1.0/unit.picosecond, \n    0.002*unit.picoseconds\n)\nfor i in range(2,3):\n    integrator.setRandomNumberSeed(i)\n    simulation_crystal.context.setPositions(PDBFile('long_test/crystal_min.pdb').getPositions())\n    simulation_crystal.context.setVelocitiesToTemperature(300*openmm.unit.kelvin)\n    simulation_crystal.reporters.clear()\n    simulation_crystal.reporters.append(app.DCDReporter(f'long_test/crystal_{i}.dcd', 1000))\n    simulation_crystal.reporters.append(app.StateDataReporter(f'long_test/crystal_{i}.csv', 1000, \n                                step=True, potentialEnergy=True, kineticEnergy=True, \n                                temperature=True, totalEnergy=True, volume=True, density=True, speed=True))\n    simulation_crystal.step(25000000)\n"

In [28]:
for i in range(2,3):
    integrator = openmm.LangevinIntegrator(
        300*unit.kelvin, 
        1.0/unit.picosecond, 
        0.002*unit.picoseconds
    )
    integrator.setRandomNumberSeed(i)
    simulation_cofold.context.setPositions(PDBFile('cofold_min.pdb').getPositions())
    simulation_cofold.context.setVelocitiesToTemperature(300*openmm.unit.kelvin)
    simulation_cofold.reporters.clear()
    simulation_cofold.reporters.append(app.DCDReporter(f'long_test/cofold_{i}.dcd', 1000))
    simulation_cofold.reporters.append(app.StateDataReporter(f'long_test/cofold_{i}.csv', 1000, 
                                step=True, potentialEnergy=True, kineticEnergy=True, 
                                temperature=True, totalEnergy=True, volume=True, density=True, speed=True))
    simulation_cofold.step(25000000)